## Bin Packing
This example solves the known hard problem of bin packing to demonstrate using a Leap hybrid CQM solver on a constrained problem with binary variables.<br>
The bin-packing problem is to pack into a number of bins of limited capacity, $c$, a collection of items. Each item, $i$, with weight, $w_i$, should be assigned to bin, $b_j$, in such a way as to minimize the number of bins used.



## Formulate the Problem
First, set up the problem parameters:
<br>$num\_items$ is the number of items. 
<br>$weights$ assigns weights, $w_i$, to each item, $i$, randomly within a configurable range, $item\_weight\_range$.
<br>$bin\_capacity$ is the bin capacity, $c$, set based on the average weight.

In [1]:
import numpy as np
num_items = 15
item_weight_range = [3, 7]
weights = list(np.random.randint(*item_weight_range, num_items))
bin_capacity = int(10 * np.mean(weights))
print("Problem: pack a total weight of {} into bins of capacity {}.".format(
      sum(weights), bin_capacity))  

Problem: pack a total weight of 64 into bins of capacity 42.


Instantiate a CQM

In [5]:
from dimod import ConstrainedQuadraticModel
cqm = ConstrainedQuadraticModel()

## Objective Function
The objective function to minimize is the number of used bins. Because a bin is either used or not, you can indicate bin usage with binary variables.<br>

Binary variable $\text{bin_used_<j>}$ indicates that bin $b_j$ is in use. The worst possible case is that each item requires an entire bin to itself, so the maximum number of bins (and the number of binary variables $\text{bin_used_<j>}$ to instantiate) is equal to the number of items, $\text{num_items}$.

In [6]:
from dimod import Binary
bin_used = [Binary(f'bin_used_{j}') for j in range(num_items)]

To minimize the number of used bins is to minimize the sum of $\text{bin_used_<j>}$ variables with value 1 (True, meaning the bin is being used):
\begin{align}
\min ( \sum_j b_j ).
\end{align}

In [12]:
cqm.set_objective(sum(bin_used))

Always keep in mind that such “variables” are actually class BinaryQuadraticModel objects, with a single variable with the requested label, $\text{bin_used_<j>}$. 

In [13]:
bin_used[0]

BinaryQuadraticModel({'bin_used_0': 1.0}, {}, 0.0, 'BINARY')

This means, for example, that multiplying by two doubles the linear bias

In [14]:
2*bin_used[0]

BinaryQuadraticModel({'bin_used_0': 2.0}, {}, 0.0, 'BINARY')

multiplying two such “variables” creates a quadratic bias


In [15]:
bin_used[0]*bin_used[1]

BinaryQuadraticModel({'bin_used_0': 0.0, 'bin_used_1': 0.0}, {('bin_used_1', 'bin_used_0'): 1.0}, 0.0, 'BINARY')

## Constraints
The bin-packing problem has two constraints: <br>
1. Each item can go into only one bin. This again is a binary outcome: item $i$ is either in bin $b_j$ or not. You can express this constraint, using binary variables, $x_{i,j}$, as

\begin{align}
\sum_j x_{i,j} = = 1.
\end{align}
 
That is, over all 
 bins, there is just one 
 with value True (or item_<i>_in_bin_<j> == 1 in the code below) for each 
.

In [16]:
item_in_bin = [[Binary(f'item_{i}_in_bin_{j}') for j in range(num_items)] for i in range(num_items)]

In [18]:
for i in range(num_items):
    one_bin_per_item = cqm.add_constraint(sum(item_in_bin[i]) == 1, label=f'item_placing_{i}')

2. Each bin has limited capacity. You can express this constraint for each bin, $b_j$, by summing over $i$ per value of $j$:

\begin{align}
\sum_j x_{i,j} \times w_j \leq c.
\end{align}

That is, for each bin $b_j$, the sum of weights for those items placed in the bin ($\text{item_<i>_in_bin_<j> == 1}$) does not exceed capacity.

In [19]:
for j in range(num_items):
    bin_up_to_capacity = cqm.add_constraint(
        sum(weights[i] * item_in_bin[i][j] for i in range(num_items)) - bin_used[j] * bin_capacity <= 0,
        label=f'capacity_bin_{j}')

For 15 items and allowing for the worst case of 15 bins, this CQM requires over 200 binary variables:

In [20]:
len(cqm.variables)

240

## Solve the Problem by Sampling
LeapHybridCQMSampler class enables you to easily incorporate Leap’s hybrid CQM solvers into your application:

In [21]:
from dwave.system import LeapHybridCQMSampler
sampler = LeapHybridCQMSampler()     

Submit the CQM to the selected solver. For one particular execution, with a maximum allowed runtime of 3 minutes, the CQM hybrid sampler returned 135 samples, out of which 131 were solutions that met all the constraints:

In [22]:
sampleset = sampler.sample_cqm(cqm,
                               time_limit=180,
                               label="SDK Examples - Bin Packing")  
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)  
if len(feasible_sampleset):      
   best = feasible_sampleset.first
   print("{} feasible solutions of {}.".format(
      len(feasible_sampleset), len(sampleset)))

131 feasible solutions of 135.


The best solution found a packing that required 2 bins:

In [23]:
selected_bins = [key for key, val in best.sample.items() if 'bin_used' in key and val]   
print("{} bins are used.".format(len(selected_bins)))     

2 bins are used.


The code below defines a simple function, $\text{get_indices}$, that returns the indices signifying the bin and item from variable names. This is used below in parsing the solutions returned from the hybrid solver.

In [24]:
def get_indices(name):
    return [int(digs) for digs in name.split('_') if digs.isdigit()]

For the best feasible solution, print the packing.

In [25]:
for bin in selected_bins:                        
    in_bin = [key for key, val in best.sample.items() if
       "_in_bin" in key and
       get_indices(key)[1] == get_indices(bin)[0]
       and val]
    b = get_indices(in_bin[0])[1]
    w = [weights[get_indices(item)[0]] for item in in_bin]
    print("Bin {} has weights {} for a total of {}.".format(b, w, sum(w)))

Bin 10 has weights [4, 4, 5, 6, 4, 3, 3, 4] for a total of 33.
Bin 8 has weights [4, 4, 4, 6, 4, 4, 5] for a total of 31.


The items were distributed in a way that kept each bin below its capacity.